### Q1. Install MLflow

In [1]:
!mlflow --version

mlflow, version 2.13.0


In [3]:
import mlflow
from mlflow.client import MlflowClient

In [3]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('homework-experiment')

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/mlruns/1', creation_time=1716905573463, experiment_id='1', last_update_time=1716905573463, lifecycle_stage='active', name='homework-experiment', tags={}>

### Q2. Download and preprocess the data

In [2]:
!pwd

/workspaces/mlops-zoomcamp/02-experiment-tracking/homework


In [3]:
!python preprocess_data.py --raw_data_path data --dest_path ./output

In [6]:
import os

print(f'How many files were saved to OUTPUT_FOLDER?: {len(os.listdir('output'))}')

How many files were saved to OUTPUT_FOLDER?: 4


### Q3. Train a model with autolog

In [22]:
!python train.py

2024/05/28 14:12:50 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/05/28 14:12:50 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

##### mlflow ui --backend-store-uri=sqlite:///mlflow.db

In [4]:
mlflow.search_runs()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.training_mean_absolute_error,metrics.training_root_mean_squared_error,metrics.training_r2_score,metrics.training_score,...,params.criterion,tags.estimator_name,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.git.commit,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.estimator_class,tags.question,tags.mlflow.source.type
0,1402d5eb1d6a443ea382aabc499e9bc7,1,RUNNING,/workspaces/mlops-zoomcamp/02-experiment-track...,2024-05-28 14:12:56.601000+00:00,None,3.42447,5.204138,0.667398,0.667398,...,squared_error,RandomForestRegressor,grandiose-rook-574,codespace,7d2b0fd81465afb373040bf05247875fcd565d9f,"[{""run_id"": ""1402d5eb1d6a443ea382aabc499e9bc7""...",train.py,sklearn.ensemble._forest.RandomForestRegressor,3,LOCAL


In [6]:
run = mlflow.search_runs(filter_string="tags.question='3'")

In [17]:
print(f'What is the value of the min_samples_split parameter: {run['params.min_samples_split'][0]}')

What is the value of the min_samples_split parameter: 2


### Q4. Launch the tracking server locally

##### mlflow server --backend-store-uri=sqlite:///mlflow.db --default-artifact-root=./artifacts

### Q5. Tune model hyperparameters

In [1]:
!python hpo.py

  0%|                                    | 0/15 [00:00<?, ?trial/s, best loss=?]/opt/conda/envs/experiment-tracking-homework-env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

  7%|▋          | 1/15 [00:08<02:01,  8.64s/trial, best loss: 5.370086069268862]/opt/conda/envs/experiment-tracking-homework-env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

 13%|█▍         | 2/15 [00:09<00:50,  3.87s/trial, best loss: 5.370086069268862]/opt/conda/envs/experiment-tracking-homework-env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in versio

In [11]:
mlflow.set_experiment('random-forest-hyperopt')
mlflow.set_tracking_uri('http://127.0.0.1:5000')

best_run = mlflow.search_runs(
    filter_string='',
    order_by=['metric.rmse ASC'],
    max_results=1
    )

In [17]:
f"What's the best validation RMSE that you got? {best_run['metrics.rmse'].values[0]:.4f}"

"What's the best validation RMSE that you got? 5.3354"